# Datasets
A brainstorm of what datasets would be useful

## By player per game
A dictionary of player's and their stats in all their games

## By team per game
A dictionary of all teams and overall stats for each game they've played

## Ratings
A dictionary of all players and their current ratings. Not on a game by game basis, but an aggregate.
Sources could include:
* NBA official stats
* 2K
* Other free stats sources

## Data structure
It might be useful to have custom data structures to access all this data. For example, I could have a specific data structure for box score, player stats, team stats.

In [43]:
import datetime as dt
from bs4 import BeautifulSoup
import requests
import re

In [74]:
# ESPN
START_DATE = dt.datetime(2018, 10, 17)
url_start = 'http://www.espn.com.au/nba/scoreboard/_/date/'

data_day = dt.datetime(2018, 10, 17)

date_string = str(data_day.year) + str(data_day.month).zfill(2) + str(data_day.day).zfill(2) # zfill for leading 0 if necessary
url = url_start + date_string

if len(date_string) != 8:
    raise ValueError('date_string needs to be in yyyymmdd format, date_string is ' + date_string)

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [87]:
# NBA
START_DATE = dt.datetime(2018, 10, 17)
url_start = 'https://au.global.nba.com/schedule/#!/'

data_day = dt.datetime(2018, 10, 17)
# zfill for leading 0 if necessary
date_string = str(data_day.year) + '-' + str(data_day.month).zfill(2) + '-' + str(data_day.day).zfill(2) 
url = url_start + date_string

if len(date_string) != 10:
    raise ValueError('date_string needs to be in yyyy-mm-dd format, date_string is ' + date_string)

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [124]:
# games = soup.findAll('a', attrs={'href': re.compile('boxscore')})
# games = soup.findAll('div', attrs={'id': 'events'})
games = soup.findAll('table', attrs={'id':'sib-league-schedule'})
assert len(games) == 1
games_table = games[0]

In [125]:
games_table

<table class="sib-zebra-stripes" id="sib-league-schedule">
<thead class="not-sortable">
<tr class="headers">
<th class="not-sortable col-time" data-ng-i18next="schedulePage.time" data-ng-show="!paginate || currentpage==1"></th>
<th class="col-matchup" data-ng-i18next="schedulePage.matchup" data-ng-show="!paginate || currentpage==1"></th>
<th class="col-score" data-ng-i18next="schedulePage.score" data-ng-show="!paginate || currentpage==1"></th>
<th class="col-location" data-ng-i18next="schedulePage.location" data-ng-show="!paginate"></th>
<th class="col-broadcaster" data-ng-i18next="schedulePage.broadcast" data-ng-show="group.status == 'Incomplete' &amp;&amp; (!paginate || currentpage==2)"></th>
<th class="col-location" data-ng-i18next="schedulePage.location" data-ng-show="paginate &amp;&amp; currentpage==2"></th>
<th class="col-links" data-ng-i18next="schedulePage.links" data-ng-show="!paginate || currentpage==1"></th>
</tr>
</thead>
<tbody data-ng-repeat="date in group.dates">
<tr cla

In [126]:
for row in games_table.findAll('tr', attrs={'data-ng-repeat':"game in date.games"}):
    print(row)
    print ('-------------------------------')

<tr data-ng-repeat="game in date.games">
<td class="col-time" data-ng-show="!paginate || currentpage==1">
<span bo-show="game.profile.scheduleCode ">{{ game.boxscore.statusDesc  }}</span>
<span bo-hide="game.profile.scheduleCode ">{{ game.profile.utcMillis | date:'shortTime' }}</span>
</td>
<td class="col-matchup" data-ng-show="!paginate || currentpage==1"><a bo-href="'/teams/#!/' + game.awayTeam.profile.code" bo-text="game.awayTeam.profile.displayAbbr" sib-team-urls="" team="game.awayTeam.profile"></a> @ <a bo-href="'/teams/#!/' + game.homeTeam.profile.code " bo-text="game.homeTeam.profile.displayAbbr" sib-team-urls="" team="game.homeTeam.profile"></a></td>
<td data-ng-show="!paginate || currentpage==1">
<span bo-show="game.boxscore.status &gt; 1"><span bo-class="{'red': game.boxscore.awayScore &gt; game.boxscore.homeScore }" bo-text=" game.boxscore.awayScore"></span> - <span bo-class="{'red': game.boxscore.awayScore &lt; game.boxscore.homeScore }" bo-text=" game.boxscore.homeScore"><